In [2]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import io
import torch
print(torch.__version__)
import torch.nn as NN
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import argparse
import time
import os
from tqdm import tqdm_notebook as tqdm

1.7.1


In [7]:
class CSVDataset(Dataset):
  def __init__(self, df, path, transform=None, balance = True):
      # loads csv file as a dataframe object
      self.df = df
      self.transform = transform
      self.path = path
      if balance:
            df_pos = df[df['class'] > 0]
            df_neg = df[df['class'] == 0]
            len_pos = len(df_pos)
            len_neg = len(df_neg)
            df_pos_new = df_pos.sample(n = len_neg, axis = 0, replace = True)
            self.df = pd.concat([df_pos_new, df_neg], axis = 0)

  def __len__(self):
      return len(self.df)

  def __getitem__(self, index):
      # load label
      lbl = self.df.iloc[index]
      img_path = lbl['filename']
      # load class
      if lbl['class'] != 0:
        img_class = 1
      else:
        img_class = 0

      # convert path to load images on HPCC
      img_path = self.path + img_path
      img = Image.open(img_path).convert('RGB')
      img = img.resize((224,224))
      
      if self.transform:
          img = self.transform(img)

      # img = np.transpose(img, (2,0,1))

      # return image and label
      # return img, torch.from_numpy(np.array([img_class])).unsqueeze(0).long()
      return img, img_class
      # return img, img_class


In [9]:
df=pd.read_csv("coordinates_train.csv")
df=df[0:1600].copy()
testTransform = transforms.Compose([
    transforms.ToTensor()
])
testDataset = CSVDataset(df, 'E:/Brandeis/CS149B/Project1/COSI-149B/project1/', testTransform)

In [4]:
os.path.exists("/project1/raw/P_NCALM_5_00334.tif")

False

In [10]:
for img, _ in testDataset:
  print(img.size())
  break;

torch.Size([3, 224, 224])


In [6]:
def getMeanStd(dataset):
  mean = [0,0,0]
  std = [0,0,0]
  num = len(dataset)
  for k in tqdm(range(num)):
    img = dataset[k][0]
    for i in range(3):
      mean[i] += img[i, :, :].mean()
      std[i] += img[i, :, :].std()
  
  mean = np.asarray(mean) / num
  std = np.asarray(std) / num
  
  print("normMean = {}".format(mean))
  print("normstdevs = {}".format(std))
  return mean, std

In [ ]:
mean, std = getMeanStd(testDataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



normMean = [0.7192409 0.7192409 0.7192409]
normstdevs = [0.01884892 0.01884892 0.01884892]



normMean = [0.7192409 0.7192409 0.7192409]
normstdevs = [0.01884892 0.01884892 0.01884892]

In [12]:
# use all
df_all = df.copy()
#shuffle data
# df_all = df_all.sample(frac=1).reset_index(drop=True)
# df_all.to_csv('data.csv')

In [13]:
# train using 90% data, rest 10% for test
size = len(df_all)
div = int(size/10*9)
df_train = df_all[0:div]
df_test = df_all[div:]

In [19]:
mean = (0.7192409, 0.7192409, 0.7192409)
std = (0.01884892, 0.01884892, 0.01884892)
transform_set = [transforms.Resize(100),
                transforms.CenterCrop(100),
                transforms.RandomRotation(degrees = 180)]
trainTransform = transforms.Compose([
    transforms.RandomChoice(transform_set),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
testTransform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
trainSet = CSVDataset(df_train, 'E:/Brandeis/CS149B/Project1/COSI-149B/project1/', trainTransform)
testSet = CSVDataset(df_test, 'E:/Brandeis/CS149B/Project1/COSI-149B/project1/', testTransform, balance = False)

In [17]:
train_batchsize = 64
test_batchsize = 10
trainEpochs = 500
learningRate = 0.0001
sgdmomentum = 0.9
betas = (0.9, 0.999)
eps = 1e-08
weight_decay=0
trainLoader = DataLoader(trainSet, batch_size=train_batchsize, shuffle=True)
testLoader = DataLoader(testSet, batch_size=test_batchsize, shuffle=False)
pos_num = 0
neg_num = 0
for _, (data, target) in enumerate(trainLoader):
    for i in range(len(target)):
        if target[i] == 1:
            pos_num = pos_num + 1;
        else:
            neg_num = neg_num + 1;
print('pos: {} neg: {}'.format(pos_num, neg_num))

pos: 1012 neg: 1012


In [11]:
use_gpu = torch.cuda.is_available()
use_gpu = False

In [12]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Mar 23 18:02:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.38       Driver Version: 456.38       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 960M   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   29C    P8    N/A /  N/A |     40MiB /  4096MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  

In [ ]:
class FocalLoss(NN.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [13]:
# Define the model
class Net(NN.Module):
    def __init__(self, num_classes=2):
      super(Net, self).__init__()
      net = models.vgg16_bn(pretrained=False) 
      net.classifier = NN.Sequential()	
      self.features = net		
      self.classifier = NN.Sequential(  
              NN.Linear(512 * 7 * 7, 4096),
              NN.ReLU(True),
              NN.Dropout(p=0.5),
              NN.Linear(4096, 4096),
              NN.ReLU(True),
              NN.Dropout(p=0.5),
              NN.Linear(4096, num_classes),
      )

    def forward(self, x):
      x = self.features(x)
      x = x.view(x.size(0), -1)
      x = self.classifier(x)
      return F.log_softmax(x, dim=1)

def train(model, train_loader, optimizer, epoch, print_batch = False):
    model.train()
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        criterion = FocalLoss(alpha = 0.25)
        if (use_gpu):
            model = model.cuda()
            data = data.cuda()
            target = target.cuda()
            criterion = criterion.cuda()
        optimizer.zero_grad()
        #output = model(torch.autograd.Variable(data))
        output = model( data )
        loss = criterion(output, target.long())
        loss.backward()
        epoch_loss = epoch_loss + loss.item()
        optimizer.step()
        if batch_idx % 100 == 0 and print_batch:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            # print(output)
    epoch_loss = (1. * epoch_loss) / len(train_loader)
    print('Train Epoch: {}\tAverageLoss: {:.6f}'.format(
    epoch+1, epoch_loss))

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    totalT = 0
    totalF = 0
    correctT = 0
    correctF = 0

    with torch.no_grad():
        for data, target in test_loader:
            if (use_gpu):
                model = model.cuda()
                data = data.cuda()
                target = target.cuda()
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            for i in range(len(target)):
              if target[i] == 1:
                totalT = totalT+1
                if target[i] == pred[i]:
                  correctT = correctT+1
              else:
                totalF = totalF+1
                if target[i] == pred[i]:
                  correctF = correctF+1


    test_loss /= len(test_loader.dataset)
    recall = 1.* correctT / totalT
    precision = 1. * correctT / (totalF - correctF)
    F1 = 2*recall*precision/(recall+precision)

    print('\nTest set: Average loss: {:.6f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print('False: {}/{} True: {}/{}\n'.format(
        correctF, totalF, correctT, totalT
    ))
    print('recall: {}\tprecision: {}\tF1: {}\n'.format(recall, precision, F1))
    summary = {}
    summary['recall'] = recall
    summary['precision'] = precision
    summary['accuracy'] = 1. * correct / len(test_loader.dataset)
    summary['F1'] = F1
                                             
    return test_loss, summary

In [15]:
#ResNet:https://blog.csdn.net/winycg/article/details/86709991
class BasicBlock(NN.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = NN.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = NN.BatchNorm2d(planes)
        self.conv2 = NN.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = NN.BatchNorm2d(planes)
        self.shortcut = NN.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = NN.Sequential(
                NN.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                NN.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(NN.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = NN.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = NN.BatchNorm2d(planes)
        self.conv2 = NN.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = NN.BatchNorm2d(planes)
        self.conv3 = NN.Conv2d(planes, self.expansion*planes,
                               kernel_size=1, bias=False)
        self.bn3 = NN.BatchNorm2d(self.expansion*planes)

        self.shortcut = NN.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = NN.Sequential(
                NN.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                NN.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(NN.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = NN.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = NN.BatchNorm2d(64)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = NN.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return NN.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2], num_classes = 2)

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3], num_classes = 2)

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3], num_classes = 2)

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3], num_classes = 2)

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3], num_classes = 2)

In [16]:
curr = time.strftime("%m%d%H", time.localtime())
# Train and test
model = ResNet18()
# model.load_state_dict(torch.load("cnn_vgg_40_4.pt"))
# optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum = sgdmomentum)
optimizer = optim.Adam(model.parameters(), lr=learningRate, betas=betas, eps=eps, weight_decay=weight_decay)
# optimizer = optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=10, verbose=True, threshold=0.001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

for epoch in range(trainEpochs):
    train(model, trainLoader, optimizer, epoch)
    loss, _ = test(model, testLoader)
    scheduler.step(loss)
    torch.save(model.state_dict(),"ResNet18" + curr + ".pt")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [57]:
torch.cuda.empty_cache()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learningRate, betas=betas, eps=eps, weight_decay=weight_decay)
# optimizer = optim.Adadelta(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True, threshold=1, threshold_mode='abs', cooldown=0, min_lr=0, eps=1e-08)

loss = 1
for i in range(10):
  print(i)
  scheduler.step(loss)

0
1
2
Epoch     3: reducing learning rate of group 0 to 5.0000e-05.
3
4
Epoch     5: reducing learning rate of group 0 to 2.5000e-05.
5
6
Epoch     7: reducing learning rate of group 0 to 1.2500e-05.
7
8
Epoch     9: reducing learning rate of group 0 to 6.2500e-06.
9
